In [1]:
import numpy as np
import pandas as pd

#vectorization imports
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import BernoulliNB
from sklearn import metrics

labels = {
    "anime": 1,
    "AskReddit": 2,
    "baseball": 3,
    "canada": 4, 
    "conspiracy": 5, 
    "europe": 6, 
    "funny": 7, 
    "gameofthrones": 8, 
    "GlobalOffensive": 9,
    "hockey" :10, 
    "leagueoflegends": 11, 
    "movies": 12, 
    "Music": 13, 
    "nba":14, 
    "nfl":15, 
    "Overwatch":16, 
    "soccer":17, 
    "trees":18, 
    "worldnews":19, 
    "wow":20
}

def k_fold(dataframe, k):
    temp = []
    folds = []
    r = dataframe.shape[0]
    q = int(r/k)
    rest = r%k

    n = 0
    for i in range(k):
        if i < rest:
            q += 1
        data = dataframe.iloc[n:n+q]
        temp.append(data)
        n += q
        if i < rest:
            q -= 1

    for i in range(k):
        data = temp[i]
        for j in range(1,k-1):
            data = data.append(temp[(j+i)%k], ignore_index=True)
        folds.append(data)
        folds.append(temp[(i+k-1)%k])
    return folds

def bnbayes(X, y):
    y = y.to_numpy()
    num_features = X.shape[1]
    num_classes = 20
    theta_k = np.zeros(num_classes)
    theta_j_k = np.zeros((num_classes, num_features))
    for k in range(num_classes):
        summ = (y==k).sum()
        y_r = y.shape[0]
        theta_k = summ/y_r
        theta_j_k[k] = (X[y==k,:].sum(axis=0)+1)/(summ+2)
    return theta_k, theta_j_k

def predict(theta_k, theta_j_k, X):
    X = X.toarray()
    XT = np.transpose(X)
    eq = np.dot(np.log(theta_j_k), XT)
    eq2 = np.dot(np.log(np.subtract(np.ones_like(theta_j_k),theta_j_k)), np.subtract(np.ones_like(XT),XT))
    prob = np.add(np.log(theta_k), np.add(eq, eq2))
    return np.argmax(prob, axis=0)

def BNB():
    train_data = pd.read_csv("reddit_train.csv")
    train_data = train_data.drop_duplicates(subset={"id"})
    train_data = train_data.drop(['id'], axis=1)
    train_data['subreddits'] = train_data['subreddits'].map(labels)
    
    k = 5
    k_folds = k_fold(train_data, k)
    
    k = 0
    scoreBNB = 0
    while (k < 9):
        X_train = k_folds[k]['comments']
        X_test = k_folds[k + 1]['comments']
        y_train = k_folds[k]['subreddits']
        y_test = k_folds[k + 1]['subreddits']

        vectorizer = CountVectorizer(binary = True, strip_accents='unicode')
        v_train = vectorizer.fit_transform(X_train)
        v_test = vectorizer.transform(X_test)
     
        theta_k, theta_j_k = bnbayes(v_train, y_train)
        y_pred = predict(theta_k, theta_j_k, v_test)
        scoreBNB = scoreBNB + metrics.accuracy_score(y_test, y_pred)
        
        #clf = BernoulliNB()
        #clf.fit(v_train, y_train)
        #y_pred = clf.predict(v_test)
        #scoreBNB = scoreBNB + metrics.accuracy_score(y_test, y_pred)

        k = k + 2
    print("BNB", scoreBNB/5)
    
BNB()

BNB 0.4479285714285714
